# 4.9 Part 1 Steps and Answers

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'C:\Users\jmowe\OneDrive\5-9-2023 Instacart Basket Analysis'

In [3]:
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data','customers.csv'), index_col = False)

In [4]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_aggd.pkl'))

### Step 4

In [5]:
# Checking the customer data

df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
df_customers.shape

(206209, 10)

In [7]:
#renaming columns for better understanding

df_customers.rename(columns = {'Surnam': 'Last Name'}, inplace = True)

In [8]:
df_customers.rename(columns = {'n_dependants': 'Number of Dependants'}, inplace = True)

In [9]:
df_customers.rename(columns = {'fam_status': 'Marital Status'}, inplace = True)

In [10]:
df_customers.head()

,user_id,First Name,Last Name,Gender,STATE,Age,date_joined,Number of Dependants,Marital Status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [11]:
#It's unclear which columns are relevant for our analysis yet. Will hold off on dropping columns.

### Step 5

In [12]:
# Fundamental data quality and consistency checks - Missing values and dups and mixed-type data

df_customers.dtypes

user_id                  int64
First Name              object
Last Name               object
Gender                  object
STATE                   object
Age                      int64
date_joined             object
Number of Dependants     int64
Marital Status          object
income                   int64
dtype: object

In [13]:
df_customers['user_id'] = df_customers['user_id'].astype('str')

In [14]:
df_customers.describe()

,Age,Number of Dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


In [15]:
df_customers.isnull().sum()

user_id                     0
First Name              11259
Last Name                   0
Gender                      0
STATE                       0
Age                         0
date_joined                 0
Number of Dependants        0
Marital Status              0
income                      0
dtype: int64

In [16]:
#observing the blank first names

df_missing = df_customers[df_customers["First Name"].isnull() == True]

In [17]:
df_missing

,user_id,First Name,Last Name,Gender,STATE,Age,date_joined,Number of Dependants,Marital Status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819
...,...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3/31/2020,3,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,4/1/2020,2,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,4/1/2020,1,married,45275
206162,187532,NaN,Floyd,Female,California,39,4/1/2020,0,single,56325


In [18]:
# Check for data consistency

for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)
  else:
    print ('No mixed dtypes', col)

No mixed dtypes user_id
First Name
No mixed dtypes Last Name
No mixed dtypes Gender
No mixed dtypes STATE
No mixed dtypes Age
No mixed dtypes date_joined
No mixed dtypes Number of Dependants
No mixed dtypes Marital Status
No mixed dtypes income


In [19]:
# First name is both missing in some values and has mixed data type entries. Solution: change First Name data type to be uniform and likely ignore the missing values, as first name is not
# too relevant to our data most likely.

df_customers['First Name'] = df_customers['First Name'].astype('str')

In [20]:
df_customers.isnull().sum()

user_id                 0
First Name              0
Last Name               0
Gender                  0
STATE                   0
Age                     0
date_joined             0
Number of Dependants    0
Marital Status          0
income                  0
dtype: int64

In [21]:
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)
  else:
    print ('No mixed dtypes', col)

No mixed dtypes user_id
No mixed dtypes First Name
No mixed dtypes Last Name
No mixed dtypes Gender
No mixed dtypes STATE
No mixed dtypes Age
No mixed dtypes date_joined
No mixed dtypes Number of Dependants
No mixed dtypes Marital Status
No mixed dtypes income


In [22]:
#checking dupilicates with a new df

df_dups = df_customers[df_customers.duplicated()]

In [23]:
df_dups.shape

(0, 10)

In [27]:
df_customers.shape

(206209, 10)

### Step 6

In [24]:
#merging customer data into main data set

ords_prods_merge.dtypes

order_id                     int64
user_id                      int64
order_number                 int64
order_day_of_week            int64
order_hour_of_day            int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
avg_price                  float64
spending_flag               object
median_freq                float64
order_freq_flag             object
dtype: object

In [25]:
# user_id is shared key. changing user_id data type to str for merging

ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('str')

In [26]:
df_combined = ords_prods_merge.merge(df_customers, on = 'user_id')

MemoryError: Unable to allocate 742. MiB for an array with shape (3, 32404859) and data type int64

In [ ]:
df_combined.head()

### Step 8

In [ ]:
#exporting

df_combined.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_combined.pkl'))